# Tutorial 2: Verbalized Distribution

There has been growing evidence that verbalized distribution can achieve high performance when asking LLMs Multiple Choice Questions. QSTN supports this option out of the box. We will show this on a simple example tp see how models predict the 2024 US election.

## Setting up the Prompt

In [ ]:
from qstn.prompt_builder import LLMPrompt
from qstn.utilities import placeholder

import pandas as pd

system_prompt = "You are an expert political analyst."

# We can add any state election we want to predict here.
elections_to_predict = [
    "2024 US Presidential Election",
    "2024 United States presidential election in Illinois",
]

# The placeholders automatically define at which point of the prompt the questions are asked.
formatted_tasks = [
    f"Please predict the outcome of the {election}. {placeholder.PROMPT_OPTIONS} {placeholder.PROMPT_AUTOMATIC_OUTPUT_INSTRUCTIONS} {placeholder.PROMPT_QUESTIONS}"
    for election in elections_to_predict
]

# If we want to ask multiple questions we can define them here or save them in a csv
questionnaire = pd.DataFrame(
    [{"questionnaire_item_id": 1, "question_content": "Percentage of each Candidate"}]
)

interviews: list[LLMPrompt] = []

# This creates a system prompt and an instruction for the model, which is not in the system prompt. We also set a seed for reproducibility.
for task, election in zip(formatted_tasks, elections_to_predict):
    interviews.append(
        LLMPrompt(
            questionnaire_source=questionnaire,
            questionnaire_name=election,
            system_prompt=system_prompt,
            prompt=task,
            seed=42,
        )
    )

/home/maxi/anaconda3/envs/qstn2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using Verbalized Distribution

To now get valid verbalized distribution output for our model we need to do two things:

1. Define the Response Generation Method.


In [ ]:
from qstn.inference.response_generation import JSONVerbalizedDistribution

# We can also adjut the automatic template to our liking. 
# If we don't want create an automatic template, we can just not put it into the prompt.
response_generation_method = JSONVerbalizedDistribution(
    output_template="Respond only in JSON format, where the keys are the names of the candidates and the values are the percentage of votes the candidate achieves.",
    output_index_only=False, # If we want to save tokens we can output only the index of our answer
)

2. Define the options the LLM should have when responding. For now we choose 5 candidates that had some chances at the end of LLamas pretraining cutoff.

In [ ]:
from qstn.prompt_builder import generate_likert_options

# Our five most likely candidates and how they are presented to the model
options = generate_likert_options(
    n=5,
    answer_texts=["Biden", "Trump", "Harris", "DeSantis", "Kennedy"],
    response_generation_method=response_generation_method,
    list_prompt_template="The candidates are {options}.", # Our automatic Option Prompt
)

Finally we have to prepare the prompt with all the options that we defined:

In [ ]:
for interview in interviews:
    interview.prepare_prompt(
        question_stem=f"Please predict the {placeholder.QUESTION_CONTENT} now. The percentage of each candidate should add up to 100%.",
        answer_options=options,
        randomized_item_order=True, # We can easily randomize the options
    )

And look at the whole prompt:

In [ ]:
system_prompt, prompt = interviews[0].get_prompt_for_questionnaire_type()

print(f"System Prompt: {system_prompt}")
print(f"Prompt: {prompt}")

System Prompt: You are an expert political analyst.
Prompt: Please predict the outcome of the 2024 US Presidential Election. The candidates are 1: Biden, 2: Trump, 3: Harris, 4: DeSantis, 5: Kennedy. Respond only in JSON format, where the keys are the names of the candidates and the values are the percentage of votes the candidate achieves. Please predict the Percentage of each Candidate now. The percentage of each candidate should add up to 100%.


And we can run inference:

In [ ]:
from vllm import LLM

# First we create the model
model = LLM("meta-llama/Llama-3.2-3B-Instruct", max_model_len=1000)

INFO 11-28 19:43:25 [utils.py:253] non-default args: {'max_model_len': 1000, 'disable_log_stats': True, 'model': 'meta-llama/Llama-3.2-3B-Instruct'}
INFO 11-28 19:43:25 [model.py:631] Resolved architecture: LlamaForCausalLM
INFO 11-28 19:43:25 [model.py:1745] Using max model len 1000
INFO 11-28 19:43:25 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.
(EngineCore_DP0 pid=3064531) INFO 11-28 19:43:26 [core.py:93] Initializing a V1 LLM engine (v0.11.2) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, structured_o

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(EngineCore_DP0 pid=3064531) INFO 11-28 19:43:27 [parallel_state.py:1208] world_size=1 rank=0 local_rank=0 distributed_init_method=tcp://134.155.63.55:42489 backend=nccl
(EngineCore_DP0 pid=3064531) INFO 11-28 19:43:27 [parallel_state.py:1394] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
(EngineCore_DP0 pid=3064531) ERROR 11-28 19:43:27 [core.py:842] EngineCore failed to start.
(EngineCore_DP0 pid=3064531) ERROR 11

(EngineCore_DP0 pid=3064531) Process EngineCore_DP0:
(EngineCore_DP0 pid=3064531) Traceback (most recent call last):
(EngineCore_DP0 pid=3064531)   File "/home/maxi/anaconda3/envs/qstn2/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
(EngineCore_DP0 pid=3064531)     self.run()
(EngineCore_DP0 pid=3064531)   File "/home/maxi/anaconda3/envs/qstn2/lib/python3.12/multiprocessing/process.py", line 108, in run
(EngineCore_DP0 pid=3064531)     self._target(*self._args, **self._kwargs)
(EngineCore_DP0 pid=3064531)   File "/home/maxi/anaconda3/envs/qstn2/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 846, in run_engine_core
(EngineCore_DP0 pid=3064531)     raise e
(EngineCore_DP0 pid=3064531)   File "/home/maxi/anaconda3/envs/qstn2/lib/python3.12/site-packages/vllm/v1/engine/core.py", line 833, in run_engine_core
(EngineCore_DP0 pid=3064531)     engine_core = EngineCoreProc(*args, **kwargs)
(EngineCore_DP0 pid=3064531)                   ^^^^^^^^^^^^^^^^^^^^^^^^^^

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

In [ ]:
from qstn.survey_manager import conduct_survey_single_item
# Second we run inference
results = conduct_survey_single_item(
    model,
    llm_prompts=interviews,
    print_conversation=True,
    seed=42,
)

## Parsing Output

We can easily parse the output now, as it is in JSON format.

In [ ]:
from qstn import parser

parsed_response = parser.parse_json(results)

We get one DataFrame for each of our Interviews.

In [ ]:
df = parsed_response[interviews[0]]
df2 = parsed_response[interviews[1]]

df

,questionnaire_item_id,question,1: Biden,2: Trump
0,1,Please predict the Percentage of each Candidat...,25,


We can also get both answers in a combined df.

In [ ]:
from qstn.utilities import create_one_dataframe

df = create_one_dataframe(parsed_response)
df